<h2>Module import</h2>

In [1]:
import threading
import ipyparallel as ipp
import subprocess
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import ttkthemes as th
import tkinter as tk
import tkinter.ttk as ttk
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from tkinter import colorchooser
import pickle
import win32api,win32con


<h2>Threading function</h2>

In [2]:
class GUI_thread():
    
    def thread(func,*args):
        t=threading.Thread(target=func,args=args)
        t.setDaemon(True)
        t.start()

<h2>Statistic function (Stochastic, options=serial or parallel computing)</h2>

In [96]:
class MC_Plot():
    
    def __init__(self):
        self.listbox=[]
        self.linecolor=['olive','g','darkorange','r','b','y']
        self.linesymbol=['-o','-s','-<','-d','-^','-p']
        self.linestyle=['-', '--', '-.', ':']
        self.linewidth_LPM=2
        self.linewidth_NPM=1
        self.linstyle_index=2
        self.Markersize=5
        self.Markevery=3

    def Datareading(self,path,Model_len,Realizations_numbers):

        import os
        import pandas as pd
        import numpy as np

        a=os.listdir(path)
        x=pd.read_table(path+a[100],header=None,encoding='gb2312',sep='\s+')
                 
        for i in range(len(x.iloc[0,:])-1):
            locals()["C"+str(i)]=np.empty([Model_len,Realizations_numbers])

        for i in range(Realizations_numbers):
            x = pd.read_table(path+a[i],header=None,encoding='gb2312',sep='\s+')
            for j in range(len(x.iloc[0,:])-1):

                locals()["C"+str(j)][:,i]=x.iloc[:,j+1]

            print('Progress= %.2f' % (i/Realizations_numbers*100)+'%', end='\r')
        print('Progress= %.2f' % 100+'%', end='\r')

        data=[]
        
        for i in range(len(x.iloc[0,:])-1):
            data.append(locals()["C"+str(i)])
            
        return list(data) 

    #------------------------------------Location mean-------------------------------------------------
    def mean(self,Data,nR):
        import numpy as np
        A=np.empty([self.__Model_len,self.__ntime]); temp=np.empty([self.__Model_len,nR])
        for i in range(self.__ntime):
            A[:,i]=np.mean(Data[:,nR*i:nR*(i+1)],axis=1)
        return A
    #-------------------Location Standard deviation------------------------------------------------
    def Standard_deviation(self,Data,nR):
        import numpy as np
        A=np.empty([self.__Model_len,self.__ntime]); temp=np.empty([self.__Model_len,nR])
        for i in range(self.__ntime):
            Mean=np.mean(Data[:,nR*i:nR*(i+1)],axis=1)
            #print(Mean)
            for j in range(nR):
                temp[:,j]=Mean
            #A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-np.mean(Data[:,nR*i:nR*(i+1)],axis=1))**2)
            A[:,i]=np.mean(np.sqrt((Data[:,nR*i:nR*(i+1)]-temp)**2),axis=1)
        return A


    #-----------------------------------Location variance-----------------------------------------------------------
    def Variance(self,Data,nR):
        import numpy as np
        A=np.empty([self.__Model_len,self.__ntime]); temp=np.empty([self.__Model_len,nR])
        for i in range(self.__ntime):
            Mean=np.mean(Data[:,nR*i:nR*(i+1)],axis=1)
            #print(Mean)
            for j in range(nR):
                temp[:,j]=Mean
            #A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-np.mean(Data[:,nR*i:nR*(i+1)],axis=1))**2)
            A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-temp)**2,axis=1)
        return A

    #----------------------------------Location Covariance-------------------------------------------------------

    def Covariance(self,Y1,Y2,nR):
        import numpy as np
        A=np.empty([self.__Model_len,self.__ntime]); temp1=np.empty([self.__Model_len,nR]); temp2=np.empty([self.__Model_len,nR])
        for i in range(self.__ntime):
            Mean1=np.mean(Y1[:,nR*i:nR*(i+1)],axis=1)
            Mean2=np.mean(Y2[:,nR*i:nR*(i+1)],axis=1)
            for j in range(nR):
                temp1[:,j]=Mean1
                temp2[:,j]=Mean2
            #A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-np.mean(Data[:,nR*i:nR*(i+1)],axis=1))**2)
            A[:,i]=np.mean((Y1[:,nR*i:nR*(i+1)]-temp1)*(Y2[:,nR*i:nR*(i+1)]-temp2),axis=1)
        return A
    
    def Covariance_Parallel(Y1,Y2,nR,Model_len,ntime):
        import numpy as np
        A=np.empty([Model_len,ntime]); temp1=np.empty([Model_len,nR]); temp2=np.empty([Model_len,nR])
        for i in range(ntime):
            Mean1=np.mean(Y1[:,nR*i:nR*(i+1)],axis=1)
            Mean2=np.mean(Y2[:,nR*i:nR*(i+1)],axis=1)
            for j in range(nR):
                temp1[:,j]=Mean1
                temp2[:,j]=Mean2
            #A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-np.mean(Data[:,nR*i:nR*(i+1)],axis=1))**2)
            A[:,i]=np.mean((Y1[:,nR*i:nR*(i+1)]-temp1)*(Y2[:,nR*i:nR*(i+1)]-temp2),axis=1)
        return A

    #----------------------------------Location Correlation Coefficient (same location)-------------------------------------
    def Correlation(self,Y1,Y2,nR):
        import numpy as np
        A=np.empty([self.__Model_len,self.__ntime]); temp1=np.empty([self.__Model_len,nR]); temp2=np.empty([self.__Model_len,nR])
        for i in range(self.__ntime):
            Mean1=np.mean(Y1[:,nR*i:nR*(i+1)],axis=1)
            Mean2=np.mean(Y2[:,nR*i:nR*(i+1)],axis=1)
            for j in range(nR):
                temp1[:,j]=Mean1
                temp2[:,j]=Mean2
            #A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-np.mean(Data[:,nR*i:nR*(i+1)],axis=1))**2)
            A[:,i]=np.mean((Y1[:,nR*i:nR*(i+1)]-temp1)*(Y2[:,nR*i:nR*(i+1)]-temp2),axis=1)/np.std(Y1[:,nR*i:nR*(i+1)],axis=1)/np.std(Y2[:,nR*i:nR*(i+1)],axis=1)

        return A
    
    def Correlation_Parallel(Y1,Y2,nR,Model_len,ntime):
        import numpy as np
        A=np.empty([Model_len,ntime]); temp1=np.empty([Model_len,nR]); temp2=np.empty([Model_len,nR])
        for i in range(ntime):
            Mean1=np.mean(Y1[:,nR*i:nR*(i+1)],axis=1)
            Mean2=np.mean(Y2[:,nR*i:nR*(i+1)],axis=1)
            for j in range(nR):
                temp1[:,j]=Mean1
                temp2[:,j]=Mean2
            #A[:,i]=np.mean((Data[:,nR*i:nR*(i+1)]-np.mean(Data[:,nR*i:nR*(i+1)],axis=1))**2)
            A[:,i]=np.mean((Y1[:,nR*i:nR*(i+1)]-temp1)*(Y2[:,nR*i:nR*(i+1)]-temp2),axis=1)/np.std(Y1[:,nR*i:nR*(i+1)],axis=1)/np.std(Y2[:,nR*i:nR*(i+1)],axis=1)

        return A   
    #----------------------------------Uncertainty------------------------------------
    def Max_Min(y):
        import numpy as np
        MAX=np.empty([len(y[0]),self.__ntime]);MIN=np.empty([len(y[0]),self.__ntime])
        for i in range(len(MAX[:][0])):
            for j in range(len(y[0])):
                MAX[j,i]=np.max(y[i][j,:])
                MIN[j,i]=np.min(y[i][j,:])
        Data=[MAX,MIN]
            #print("X=",len(MAX[:][0]))
        return Data

    def Confidence_intervals(y_mean,std_y):
        import numpy as np
        R_upper=np.empty([len(y_mean),3]);R_down=np.empty([len(y_mean),3])
        for i in range(len(R_upper[:][0])):
            for j in range(len(y_mean)):
                if i==0:
                    R_upper[j,i]=y_mean[j]+std_y[j]
                    R_down[j,i]=y_mean[j]-std_y[j]
                if i==1:
                    R_upper[j,i]=y_mean[j]+2*std_y[j]
                    R_down[j,i]=y_mean[j]-2*std_y[j]
                if i==2:
                    R_upper[j,i]=y_mean[j]+3*std_y[j]
                    R_down[j,i]=y_mean[j]-3*std_y[j]
        return [R_upper, R_down]            

    #--------------------------------------Spatial Plotting--------------------------------------------------------

    def Plotting(IPM,DPM,t_list):

        import numpy as np
        import matplotlib.pyplot as plt
        plt.switch_backend('agg')

        #-----------------------------------Parameter setting-------------------------------------
        legend=[]
        tc=['5','25','50','125','150','200','300','400','500','800']
        t=[0,0,0,0,0,0,0,0,0]
        I=["LPM 5a","LPM 25a","LPM 50a","LPM 125a","LPM 150a","LPM 200a","LPM 300a","LPM 400a","LPM 500a"]
        D=["NPM 5a","NPM 25a","NPM 50a","NPM 125a","NPM 150a","NPM 200a","NPM 300a","NPM 400a","NPM 500a"]
        color=['olive','g','darkorange','r','b','y']
        
        Isymbol=['o','s','<','d','^','o','s','<','d']
        Dsymbol=['-o','-s','-<','-d','-^','-o','-s','-<','-d']
        x=np.linspace(0,self.__Model_len-1,self.__Model_len)
        #------------------------------------Plotting(Spatial domain)---------------------------------------------- 
        
        for i in range(len(tc)):
            if t_list.count(tc[i])==1:
                t[i]=1
        count=0
        symbol=0
        for i in range(len(t)):
            if t[i]==1:
                ax,=plt.plot(x,IPM[:,count],color=color[symbol],linestyle='-.',linewidth='2',markeredgecolor='k')
                legend.append(I[count])
                symbol=symbol+1
            count=count+1

        count=0
        symbol=0
        for item in t:
            if item==1:
                ax,=plt.plot(x,DPM[:,count],Dsymbol[symbol],linewidth='1',color='k',markerfacecolor=color[symbol],markeredgecolor='k',markersize='8',markevery=3)
                legend.append(D[count])
                symbol=symbol+1
            count=count+1


        plt.legend(legend,ncol=2,fontsize='14',edgecolor='k')
        plt.xlabel('Distance(m)',fontsize='28')
        plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0),useMathText=True,useOffset=False)
        plt.rc('font', size=17)
        plt.xticks(fontsize=25),plt.yticks(fontsize=25)
        
    def Cavanas_Plot(self,canvas_n):
        
        import numpy as np
        import matplotlib.pyplot as plt
        import time
        time.sleep(0.1)
        t_list=TimeEntry.get().split(',')
        plotlist=[self.D_mean,self.P_mean,self.T_mean,self.V_mean,self.D_var,self.P_var,self.T_var,self.V_var,
                 self.DP_cov,self.HP_cov,self.DH_cov,self.DT_cov,self.TP_cov,self.DH_corr,self.HP_corr,
                 self.DP_corr,self.DT_corr,self.TP_corr]
        plotlist2=[self.D2_mean,self.P2_mean,self.T2_mean,self.V2_mean,self.D2_var,self.P2_var,self.T2_var,self.V2_var,
                 self.DP2_cov,self.HP2_cov,self.DH2_cov,self.DT2_cov,self.TP2_cov,self.DH2_corr,self.HP2_corr,
                 self.DP2_corr,self.DT2_corr,self.TP2_corr]
    
        if canvas_n==f1:
            index=int(listbox.curselection()[0])
            IPM=plotlist[index]
            DPM=plotlist2[index]
            #f1=fig.add_subplot(111,title=self.listbox[index],xlabel='Distance (m)',ylabel='Mean')
        elif canvas_n==f2:
            index2=int(listbox2.curselection()[0])
            IPM=plotlist[index2]
            DPM=plotlist2[index2]
            #f2=fig2.add_subplot(111,title=self.listbox[index],xlabel='Distance (m)',ylabel='Mean')
        
        #-----------------------------------Parameter setting-------------------------------------
        legend=[]
        tc=['5','25','50','125','150','200','300','400','500','800']
        t=[0,0,0,0,0,0,0,0,0]
        I=["LPM 5a","LPM 25a","LPM 50a","LPM 125a","LPM 150a","LPM 200a","LPM 300a","LPM 400a","LPM 500a"]
        D=["NPM 5a","NPM 25a","NPM 50a","NPM 125a","NPM 150a","NPM 200a","NPM 300a","NPM 400a","NPM 500a"]
        color=self.linecolor
        linestyle=self.linestyle
        Dsymbol=self.linesymbol
        x=np.linspace(0,self.__Model_len-1,self.__Model_len)


        #------------------------------------Plotting(Spatial domain)---------------------------------------------- 
        for i in range(len(tc)):
            if t_list.count(tc[i])==1:
                t[i]=1
        count=0
        symbol=0
        for i in range(len(t)):
            
            if t[i]==1:
                Line_color=self.linecolor[symbol]
                if Line_color.count(',')>0:
                    Line_color=Line_color.split(',')
                    Line_color=[float(x)/255 for x in Line_color]
                canvas_n.plot(x,IPM[:,count],color=Line_color,linestyle=self.linestyle[self.linstyle_index],linewidth=self.linewidth_LPM)
                legend.append(I[count])
                symbol=symbol+1
            count=count+1
        
        count=0
        symbol=0
        for item in t:
            if item==1:
                Line_color=self.linecolor[symbol]
                if Line_color.count(',')>0:
                    Line_color=Line_color.split(',')
                    Line_color=[float(x)/255 for x in Line_color]
                canvas_n.plot(x,DPM[:,count],self.linesymbol[symbol],linewidth=self.linewidth_NPM,color='k',markerfacecolor=Line_color,
                              markeredgecolor='k',markersize=self.Markersize,markevery=self.Markevery)
                legend.append(D[count])
                symbol=symbol+1
            count=count+1

        canvas_n.grid(alpha=0.3)
        canvas_n.legend(legend,ncol=2,fontsize='9',edgecolor='k')
        plt.rc('font', size=9)
        if canvas_n==f1:
            canvas.draw_idle()
        elif canvas_n==f2:
            canvas2.draw_idle()


    def button_event_Data(self):
        
        import os
        import numpy as np
        import pandas as pd
        import time
        W1.save_checker=False
        self.judge=0
    
        nR=int(nREntry.get())
        
        start_time=time.time()
        messagelistbox.insert(tk.END,"Number of realizations calculated for: "+str(nR)+" ("+str(time.asctime())+")\n")
 

        mybutton_Data['state'] ="disabled"
        mybutton_Plot['state']="disabled"
        mybutton_Parallel['state']="disabled"
        mybutton_Parallel2['state']="disabled"
        
        Data_Label["text"]='Processing'
        Data_Label["bg"]='red'
        # Data path
        IPM_Folderpath=IPMPathEntry.get()
        DPM_Folderpath=DPMPathEntry.get()
        # File path for IPM
        D_path=IPM_Folderpath+'\\Displacement\\'
        P_path=IPM_Folderpath+'\\Pressure\\'
        T_path=IPM_Folderpath+'\\Temperature\\'
        V_path=IPM_Folderpath+'\\Volumetric_strain\\'
        H_path=IPM_Folderpath+'\\Hydraulic\\'
        kc_path=IPM_Folderpath+'\\HeatCouductivity\\'
        Cp_path=IPM_Folderpath+'\\HeatCapacity\\'
        aT_path=IPM_Folderpath+'\\ThermalExpansion\\'
        E_path=IPM_Folderpath+"\\Young's_modulus\\"
        
        # File path for DPM
        D2_path=DPM_Folderpath+'\\Displacement\\'
        P2_path=DPM_Folderpath+'\\Pressure\\'
        T2_path=DPM_Folderpath+'\\Temperature\\'
        V2_path=DPM_Folderpath+'\\Volumetric_strain\\'
        H2_path=DPM_Folderpath+'\\Hydraulic\\'
        kc2_path=DPM_Folderpath+'\\HeatCouductivity\\'
        Cp2_path=DPM_Folderpath+'\\HeatCapacity\\'
        aT2_path=DPM_Folderpath+'\\ThermalExpansion\\'
        E2_path=DPM_Folderpath+"\\Young's_modulus\\"

        # Detect the data format (Model length, time length)
        
        a=os.listdir(D_path)
        z=pd.read_table(D_path+a[0],header=None,encoding='gb2312',sep='\s+')
        
        self.__Model_len=np.shape(z)[0]
        self.__ntime=np.shape(z)[1]-1
        
        
        D=np.zeros([self.__Model_len,nR*self.__ntime]);D2=np.zeros([self.__Model_len,nR*self.__ntime])
        P=np.zeros([self.__Model_len,nR*self.__ntime]);P2=np.zeros([self.__Model_len,nR*self.__ntime])
        T=np.zeros([self.__Model_len,nR*self.__ntime]);T2=np.zeros([self.__Model_len,nR*self.__ntime])
        V=np.zeros([self.__Model_len,nR*self.__ntime]);V2=np.zeros([self.__Model_len,nR*self.__ntime])
        H=np.zeros([self.__Model_len,nR*self.__ntime]);H2=np.zeros([self.__Model_len,nR*self.__ntime])
        
        #-----------------Single core mode------------------------------------------------
        if int(Select.get())==1:  
  
            D_temp=self.Datareading(D_path,self.__Model_len,nR)
            P_temp=self.Datareading(P_path,self.__Model_len,nR)
            T_temp=self.Datareading(T_path,self.__Model_len,nR)
            V_temp=self.Datareading(V_path,self.__Model_len,nR)
            #H_temp=self.Datareading(H_path,self.__Model_len,nR,H_rdata,self.check)
            
            if RV_check.get()==0:
                H_temp=self.Datareading(H_path,self.__Model_len,nR)
                H_temp=np.log(H_temp)
                self.RV_name="Hydraulic conductivity"
            elif RV_check.get()==1:
                H_temp=self.Datareading(kc_path,self.__Model_len,nR)
                self.RV_name="Thermal conductivity"
            elif RV_check.get()==2:
                H_temp=self.Datareading(Cp_path,self.__Model_len,nR)
                self.RV_name="Specific Heat"
            elif RV_check.get()==3:
                H_temp=self.Datareading(aT_path,self.__Model_len,nR)
                self.RV_name="Thermal Expansion coefficient"
            elif RV_check.get()==4:
                H_temp=self.Datareading(E_path,self.__Model_len,nR)
                self.RV_name="Young's modulus"
            
            # Data reading (DPM)
            D2_temp=self.Datareading(D2_path,self.__Model_len,nR)
            P2_temp=self.Datareading(P2_path,self.__Model_len,nR)
            T2_temp=self.Datareading(T2_path,self.__Model_len,nR)
            V2_temp=self.Datareading(V2_path,self.__Model_len,nR)
            
            if RV_check.get()==0:
                H2_temp=self.Datareading(H2_path,self.__Model_len,nR)
                H2_temp=np.log(H2_temp)
            elif RV_check.get()==1:
                H2_temp=self.Datareading(kc2_path,self.__Model_len,nR)
            elif RV_check.get()==2:
                H2_temp=self.Datareading(Cp2_path,self.__Model_len,nR)
            elif RV_check.get()==3:
                H2_temp=self.Datareading(aT2_path,self.__Model_len,nR)
            elif RV_check.get()==4:
                H2_temp=self.Datareading(E2_path,self.__Model_len,nR)
            
            for i in range(self.__ntime):

                D[:,nR*i:nR*(i+1)]=D_temp[:][i]
                P[:,nR*i:nR*(i+1)]=P_temp[:][i]
                V[:,nR*i:nR*(i+1)]=V_temp[:][i]
                H[:,nR*i:nR*(i+1)]=H_temp[:][i]

                D2[:,nR*i:nR*(i+1)]=D2_temp[:][i]
                P2[:,nR*i:nR*(i+1)]=P2_temp[:][i]
                V2[:,nR*i:nR*(i+1)]=V2_temp[:][i]
                H2[:,nR*i:nR*(i+1)]=H2_temp[:][i]
                
                T[:,nR*i:nR*(i+1)]=T_temp[:][i]
                T2[:,nR*i:nR*(i+1)]=T2_temp[:][i]
            
            # mean
            self.D_mean=self.mean(D,nR);self.D2_mean=self.mean(D2,nR)
            self.P_mean=self.mean(P,nR);self.P2_mean=self.mean(P2,nR)
            self.T_mean=self.mean(T,nR);self.T2_mean=self.mean(T2,nR)
            self.V_mean=self.mean(V,nR);self.V2_mean=self.mean(V2,nR)

            # Standard deviation
            self.D_std=self.Standard_deviation(D,nR);self.D2_std=self.Standard_deviation(D2,nR)
            self.P_std=self.Standard_deviation(P,nR);self.P2_std=self.Standard_deviation(P2,nR)
            self.T_std=self.Standard_deviation(T,nR);self.T2_std=self.Standard_deviation(T2,nR)
            self.V_std=self.Standard_deviation(V,nR);self.V2_std=self.Standard_deviation(V2,nR)

            # Variance
            self.D_var=self.Variance(D,nR);self.D2_var=self.Variance(D2,nR)
            self.P_var=self.Variance(P,nR);self.P2_var=self.Variance(P2,nR)
            self.T_var=self.Variance(T,nR);self.T2_var=self.Variance(T2,nR)
            self.V_var=self.Variance(V,nR);self.V2_var=self.Variance(V2,nR)

            # Covariance
            self.DP_cov=self.Covariance(D,P,nR);self.DP2_cov=self.Covariance(D2,P2,nR)
            self.DH_cov=self.Covariance(D,H,nR);self.DH2_cov=self.Covariance(D2,H2,nR)
            self.HP_cov=self.Covariance(H,P,nR);self.HP2_cov=self.Covariance(H2,P2,nR)
            self.DT_cov=self.Covariance(D,T,nR);self.DT2_cov=self.Covariance(D2,T2,nR)
            self.TP_cov=self.Covariance(T,P,nR);self.TP2_cov=self.Covariance(T2,P2,nR)

            #Correlation
            self.DP_corr=self.Correlation(D,P,nR);self.DP2_corr=self.Correlation(D2,P2,nR)
            self.DH_corr=self.Correlation(D,H,nR);self.DH2_corr=self.Correlation(D2,H2,nR)
            self.HP_corr=self.Correlation(H,P,nR);self.HP2_corr=self.Correlation(H2,P2,nR)
            self.DT_corr=self.Correlation(D,T,nR);self.DT2_corr=self.Correlation(D2,T2,nR)
            self.TP_corr=self.Correlation(T,P,nR);self.TP2_corr=self.Correlation(T2,P2,nR)            
        #----------------Multi core mode------------------------------------------------
        else:
    
            import ipyparallel as ipp
            import os
            
            c=ipp.Client()
            Core_number=int(Select.get())
            Task=c[:Core_number]

            # Data Reading (Parallel) 
                  
            if RV_check.get()==0:
                RV_path=H_path
                RV2_path=H2_path
            elif RV_check.get()==1:
                RV_path=kc_path
                RV2_path=kc2_path
            elif RV_check.get()==2:
                RV_path=Cp_path
                RV2_path=Cp2_path
            elif RV_check.get()==3:
                RV_path=aT_path
                RV2_path=aT2_path
            elif RV_check.get()==4:
                RV_path=E_path
                RV2_path=E2_path
            
            SyncList=[D_path,P_path,V_path,RV_path,D2_path,P2_path,V2_path,RV2_path]
            ML=[]
            RL=[]
            CL=[]
            AdopterL=[]


            for i in range(len(SyncList)):
                AdopterL.append(0)
                ML.append(self.__Model_len)
                RL.append(nR)
  

            a=Task.map_sync(MC_Plot.Datareading, AdopterL,SyncList,ML,RL)
            b=Task.map_sync(MC_Plot.Datareading, AdopterL,[T_path,T2_path],ML,RL)

            for i in range(self.__ntime):

                D[:,nR*i:nR*(i+1)]=(a[0][:])[:][i]
                P[:,nR*i:nR*(i+1)]=(a[1][:])[:][i]
                V[:,nR*i:nR*(i+1)]=(a[2][:])[:][i]
                H[:,nR*i:nR*(i+1)]=(a[3][:])[:][i]

                D2[:,nR*i:nR*(i+1)]=(a[4][:])[:][i]
                P2[:,nR*i:nR*(i+1)]=(a[5][:])[:][i]
                V2[:,nR*i:nR*(i+1)]=(a[6][:])[:][i]
                H2[:,nR*i:nR*(i+1)]=(a[7][:])[:][i]

                T[:,nR*i:nR*(i+1)]=(b[0][:])[:][i]
                T2[:,nR*i:nR*(i+1)]=(b[1][:])[:][i]

            # Transform to natural log Hydraulic

            H=np.log(H)
            H2=np.log(H2)

            # mean
            self.D_mean=self.mean(D,nR);self.D2_mean=self.mean(D2,nR)
            self.P_mean=self.mean(P,nR);self.P2_mean=self.mean(P2,nR)
            self.T_mean=self.mean(T,nR);self.T2_mean=self.mean(T2,nR)
            self.V_mean=self.mean(V,nR);self.V2_mean=self.mean(V2,nR)

            # Standard deviation
            self.D_std=self.Standard_deviation(D,nR);self.D2_std=self.Standard_deviation(D2,nR)
            self.P_std=self.Standard_deviation(P,nR);self.P2_std=self.Standard_deviation(P2,nR)
            self.T_std=self.Standard_deviation(T,nR);self.T2_std=self.Standard_deviation(T2,nR)
            self.V_std=self.Standard_deviation(V,nR);self.V2_std=self.Standard_deviation(V2,nR)

            # Variance
            self.D_var=self.Variance(D,nR);self.D2_var=self.Variance(D2,nR)
            self.P_var=self.Variance(P,nR);self.P2_var=self.Variance(P2,nR)
            self.T_var=self.Variance(T,nR);self.T2_var=self.Variance(T2,nR)
            self.V_var=self.Variance(V,nR);self.V2_var=self.Variance(V2,nR)

            # Covariance (Parallel)

            SyncList1=[D,D,H,D2,D2,H2,D,D2,T,T2]
            SyncList2=[P,H,P,P2,H2,P2,T,T2,P,P2]
            nR_list=[]
            Model_lenL=[]
            n_timeL=[]
            
            for i in range(len(SyncList1)):
                nR_list.append(nR)
                Model_lenL.append(self.__Model_len)
                n_timeL.append(self.__ntime)
                
            L_Cov=Task.map_sync(MC_Plot.Covariance_Parallel,SyncList1,SyncList2,nR_list,Model_lenL,n_timeL)

            self.DP_cov=L_Cov[0][:];self.DP2_cov=L_Cov[3][:]
            self.DH_cov=L_Cov[1][:];self.DH2_cov=L_Cov[4][:]
            self.HP_cov=L_Cov[2][:];self.HP2_cov=L_Cov[5][:]
            self.DT_cov=L_Cov[6][:];self.DT2_cov=L_Cov[7][:]
            self.TP_cov=L_Cov[8][:];self.TP2_cov=L_Cov[9][:]

            #Correlation (Parallel)

            L_Corr=Task.map_sync(MC_Plot.Correlation_Parallel,SyncList1,SyncList2,nR_list,Model_lenL,n_timeL)

            self.DP_corr=L_Corr[0][:];self.DP2_corr=L_Corr[3][:]
            self.DH_corr=L_Corr[1][:];self.DH2_corr=L_Corr[4][:]
            self.HP_corr=L_Corr[2][:];self.HP2_corr=L_Corr[5][:]
            self.DT_corr=L_Corr[6][:];self.DT2_corr=L_Corr[7][:]
            self.TP_corr=L_Corr[8][:];self.TP2_corr=L_Corr[9][:]
            
        PlotList=["Mean_Displacement","Mean_Pressure","Mean_Temperature","Mean_Volumetricstrain",
                  "C(u,u)","C(P,P)","C(v,v)","C(u,P)","C(y,P)","C(y,u)","C(u,T)","C(T,P)",
                 "Corr(u,u)","Corr(P,P)","Corr(v,v)","Corr(u,P)","Corr(y,P)","Corr(y,u)",
                 "Corr(u,T)","Corr(T,P)"]
        
        for item in PlotList:
            listbox.insert(tk.END,item)
            listbox2.insert(tk.END,item)
            self.listbox.append(item)

        mybutton_Data['state'] ="normal"
        mybutton_Plot['state']="normal"
        mybutton_Parallel['state']="normal"
        mybutton_Parallel2['state']="normal"

        self.judge=1
        
        end_time=time.time()
        messagelistbox.insert(tk.END,"Computation finish ("+str(time.asctime())+")\n")
        messagelistbox.insert(tk.END,"Last computation time: "+str(round(end_time-start_time,2))+" s\n")

        
    def button_event_Plot(self):
        import time
        time.sleep(1)
        start_time=time.time()
        plt.switch_backend('agg')
        messagelistbox.insert(tk.END,"Start figure plotting ("+str(time.asctime())+")\n")
        
        fdpi=int(Figuredpi.get())
        self.ProgressCounter=0
        self.judge=0
        Progresslen=16
        #-----------------------------------Mean-------------------------------------
        mybutton_Data['state'] = "disabled"
        mybutton_Plot['state']="disabled"
        mybutton_Parallel['state']="disabled"
        mybutton_Parallel2['state']="disabled"
        
        figdir=FigureSavePath.get()+"\\"
        t_list=TimeEntry.get().split(',')
        
        plt.figure(num=0,figsize=(10,8))
        MC_Plot.Plotting(self.D_mean,self.D2_mean,t_list)

        plt.figure(num=1,figsize=(10,8))
        MC_Plot.Plotting(self.D_mean,self.D2_mean,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Displacement (m)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'D.png',dpi=fdpi)
        progressFrame.update_idletasks()
        progress['value']=self.ProgressCounter/Progresslen*100
        
        plt.figure(num=2,figsize=(10,8))
        MC_Plot.Plotting(self.P_mean,self.P2_mean,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Pressure (Pa)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'P.png',dpi=fdpi)
        progressFrame.update_idletasks()
        progress['value']=self.ProgressCounter/Progresslen*100

        plt.figure(num=3,figsize=(10,8))
        MC_Plot.Plotting(self.T_mean,self.T2_mean,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Temperature (K)',fontsize='28')
        plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'T.png',dpi=fdpi)

        plt.figure(num=4,figsize=(10,8))
        MC_Plot.Plotting(self.V_mean,self.V2_mean,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Volumetric strain (-)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'V.png',dpi=fdpi)

        #-----------------------------------Variance------------------------------------
        plt.figure(num=5,figsize=(10,8))
        MC_Plot.Plotting(self.D_var,self.D2_var,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(u$_{x}$,u$_{x}$)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'C(u,u).png',dpi=fdpi)

        plt.figure(num=6,figsize=(10,8))
        MC_Plot.Plotting(self.P_var,self.P2_var,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(P,P)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'C(P,P).png',dpi=fdpi)

        plt.figure(num=7,figsize=(10,8))
        MC_Plot.Plotting(self.T_var,self.T2_var,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(T,T)',fontsize='28')
        plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'C(T,T).png',dpi=fdpi)

        plt.figure(num=8,figsize=(10,8))
        MC_Plot.Plotting(self.V_var,self.V2_var,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(v,v)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'C(v,v).png',dpi=fdpi)

        #-------------------------------------------Covariance-----------------------------------

        plt.figure(num=9,figsize=(9,7))
        MC_Plot.Plotting(self.DP_cov,self.DP2_cov,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(u$_{x}$,P)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'C(u,P).png',dpi=fdpi)

        plt.figure(num=10,figsize=(9,7))
        MC_Plot.Plotting(self.HP_cov,self.HP2_cov,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(y,P)',fontsize='28')
        plt.tight_layout()
        plt.savefig(figdir+'C(y,P).png',dpi=fdpi)

        plt.figure(num=11,figsize=(9,7))
        MC_Plot.Plotting(self.DH_cov,self.DH2_cov,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(y,u$_{x}$)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'C(y,u).png',dpi=fdpi)

        plt.figure(num=12,figsize=(9,7))
        MC_Plot.Plotting(self.DT_cov,self.DT2_cov,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(u$_{x}$,T)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'C(u,T).png',dpi=fdpi)

        plt.figure(num=13,figsize=(9,7))
        MC_Plot.Plotting(self.TP_cov,self.TP2_cov,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('C(T,P)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'C(T,P).png',dpi=fdpi)

        #----------------------------------------------Correlation Coefficient----------------------------------------

        plt.figure(num=110,figsize=(9,7))
        MC_Plot.Plotting(self.DH_corr,self.DH2_corr,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Corr(y,u$_{x}$)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'Corr(y,u).png',dpi=fdpi)

        plt.figure(num=111,figsize=(9,7))
        MC_Plot.Plotting(self.HP_corr,self.HP2_corr,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Corr(y,P)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'Corr(y,P).png',dpi=fdpi)

        plt.figure(num=112,figsize=(9,7))
        MC_Plot.Plotting(self.DP_corr,self.DP2_corr,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Corr(u$_{x}$,P)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'Corr(u,P).png',dpi=fdpi)

        plt.figure(num=113,figsize=(9,7))
        MC_Plot.Plotting(self.DT_corr,self.DT2_corr,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Corr(u$_{x}$,T)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'Corr(u,T).png',dpi=fdpi)

        plt.figure(num=114,figsize=(9,7))
        MC_Plot.Plotting(self.TP_corr,self.TP2_corr,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Corr(T,P)',fontsize='28')
        #plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'Corr(T,P).png',dpi=fdpi)
    
        plt.close('all')
        
        mybutton_Data['state'] = "normal"
        mybutton_Plot['state']="normal"
        mybutton_Parallel['state']="normal"
        mybutton_Parallel2['state']="normal"
        
        end_time=time.time()
        
        messagelistbox.insert(tk.END,"Plot complete ("+str(time.asctime())+")\n")
        messagelistbox.insert(tk.END,"Last computation time: "+str(round(end_time-start_time,2))+" s\n")
        
        self.judge=1

        
    def button_Parallel(self):
        
        mybutton_Data['state'] = "disabled"
        mybutton_Parallel['state']="disabled"
        mybutton_Parallel2['state']="disabled"
        import time
        subprocess.Popen(["ipcluster", "start"])
        button_Parallel_label['bg']='Lightskyblue'
        button_Parallel_label['fg']='red'
        #button_Parallel1['font']='5'
        for i in range(10):
            button_Parallel_label['text']='Engine will start in '+str(10-i)+' s\n'+"Don't shut down the app"
            time.sleep(1)
        button_Parallel_label['text']='Multi-Core mode'
        button_Parallel_label['bg']='light pink' 
        button_Parallel_label['fg']='black'
        c=ipp.Client()
        Select['values'] = c.ids
        c[0].push({"Case1":Case1})
        mybutton_Data['state'] = "normal"
        mybutton_Parallel['state']="normal"
        mybutton_Parallel2['state']="normal"
        
    def button_Parallel_stop(self):
        
        subprocess.Popen(["ipcluster", "stop"])
        
        button_Parallel_label['text']='Single-Core mode'
        button_Parallel_label['bg']='blue'
        button_Parallel_label['fg']='white'
        Select['values'] = '1'
        Select.current(0)
        
    def Progress(self):
        import time 
        time.sleep(1.5)
        while self.judge==0:
            Data_Label['text']='Processing.'
            time.sleep(0.2)
            Data_Label['text']='Processing..'
            time.sleep(0.2)
            Data_Label['text']='Processing...'
            time.sleep(0.2)
            Data_Label['text']='Processing....'
            time.sleep(0.2)
            Data_Label['text']='Processing.....'
        Data_Label["text"]="Data ready"
        Data_Label["bg"]='orange'
        

<h2>Statistic function (Deterministic)</h2>

In [97]:
 
class Dt_Plot():       
    #------------------------------------Reading data (DataFrame)----------------------------------------------------

    def Datareading(path):
        import pandas as pd
        data = pd.read_table(path,header=None,encoding='gb2312',sep='\s+')
        return data

    #--------------------------------------Spatial Plotting--------------------------------------------------------

    def Plotting(IPM,DPM,t_list):
        #-----------------------------------Parameter setting-------------------------------------
        import pandas as pd 
        import matplotlib.pyplot as plt
        
        legend=[]
        tc=['5','25','50','125','150','200','300','400','500','800']
        t=[0,0,0,0,0,0,0,0,0,0]
        I=["0","LPM 5a","LPM 25a","LPM 50a","LPM 125a","LPM 150a","LPM 200a","LPM 300a","LPM 400a","LPM 500a","LPM 800a"]
        D=["0","NPM 5a","NPM 25a","NPM 50a","NPM 125a","NPM 150a","NPM 200a","NPM 300a","NPM 400a","NPM 500a","NPM 800a"]
        color=['olive','g','darkorange','r','b','y','m','purple','c','gray','brown']
        Isymbol=['o','s','<','d','^','o','s','<','d','o','s','<']
        Dsymbol=['o','s','<','d','^','o','s','<','d','o','s','<']
        ncol=2
        
        for i in range(len(tc)):
            if t_list.count(tc[i])==1:
                t[i]=1

        #------------------------------------Plotting(Spatial domain)---------------------------------------------- 
        count=1 
        symbol=0
        for item in t:
            if item==1:
                plt.plot(IPM.iloc[:,0],IPM.iloc[:,count],color=color[symbol],linewidth='2',markeredgecolor='k')
                legend.append(I[count])
                symbol=symbol+1
            count=count+1
        count=1
        symbol=0
        for item in t:
            if item==1:
                plt.plot(DPM.iloc[:,0],DPM.iloc[:,count],Dsymbol[symbol],linewidth='2',markerfacecolor=color[symbol],markeredgecolor='k',markersize='8',markevery=3)
                legend.append(D[count])
                symbol=symbol+1
            count=count+1
        if len(t_list)>=6:
            ncol=3
        plt.legend(legend,ncol=ncol,fontsize='14',edgecolor='k')
        plt.xlabel('Distance(m)',fontsize='28')
        plt.xticks(fontsize=25),plt.yticks(fontsize=25)
        plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0),useMathText=True,useOffset=False)
        plt.rc('font', size=17)


    #-------------------------------------------Temporal Plotting-------------------------------------------------

    def Plotting_time(IPM,DPM,location='best'):
        #-----------------------------------Parameter setting-------------------------------------
        legend=[]
        I=['LPM 11m','LPM 37m','LPM 75m','LPM 113m']
        D=['NPM 11m','NPM 37m','NPM 75m','NPM 113m']
        color=['olive','g','darkorange','r','b','y']
        Isymbol=['o','s','<','d','^','o','s','<','d']
        Dsymbol=['o','s','<','d','^','o','s','<','d']
        #------------------------------------Plotting(Spatial domain)---------------------------------------------- 
        count=1 
        symbol=0
        for i in range(4):
            plt.plot(IPM.iloc[:,0],IPM.iloc[:,count],color=color[symbol],linewidth='2',markeredgecolor='k')
            legend.append(I[symbol])
            symbol=symbol+1
            count=count+1
        count=1
        symbol=0
        for i in range(4):
            plt.plot(DPM.iloc[:,0],DPM.iloc[:,count],Dsymbol[symbol],linewidth='2',markerfacecolor=color[symbol],markeredgecolor='k',markersize='8',markevery=3)
            legend.append(D[symbol])
            symbol=symbol+1
            count=count+1

        plt.legend(legend,ncol=2,fontsize='17',edgecolor='k',loc=location)
        plt.xlabel('Time(d)',fontsize='28')
        plt.xticks(fontsize=25),plt.yticks(fontsize=25)
        plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0),useMathText=True,useOffset=False)
        plt.rc('font', size=17)

    def Plotting_differ(Data,t1,t2,t3,t4,t5,t6):
        #-----------------------------------Parameter setting-------------------------------------
        legend=[]
        t=[t1,t2,t3,t4,t5,t6]
        I=["0","5a","25a","50a","150a","300a","500a"]
        #D=["0","NPM 1hr","NPM 5hr","NPM 10hr","NPM 30hr","NPM 50hr","NPM 1d","NPM 5d","NPM 15d","NPM 25d","NPM 100d","NPM 200d","NPM 300d"]
        color=['olive','g','darkorange','r','b','y']
        Isymbol=['o','s','<','d','^','o','s','<','d']
        Dsymbol=['o','s','<','d','^','o','s','<','d']
        #------------------------------------Plotting(Spatial domain)---------------------------------------------- 
        count=1 
        symbol=0
        for item in t:
            if item==1:
                ax,=plt.plot(np.linspace(0,150,151),Data.iloc[:,count],color=color[symbol],linewidth='2',markeredgecolor='k')
                legend.append(I[count])
                symbol=symbol+1
            count=count+1
        count=1
        symbol=0


        plt.legend(legend,ncol=2,fontsize='14',edgecolor='k')
        plt.xlabel('Distance(m)',fontsize='28')
        plt.xticks(fontsize=25),plt.yticks(fontsize=25)
        #plt.ticklabel_format(style='sci',axis='y',scilimits=(0,0),useMathText=True,useOffset=False)
        plt.rc('font', size=17)
        ax=plt.gca()
        vals = ax.get_yticks()
        ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
        
    def Cavanas_Plot(self,canvas_n):
        
        import numpy as np
        import matplotlib.pyplot as plt
        import time
        time.sleep(0.1)
        t_list=TimeEntry.get().split(',')
        plotlist=[self.D_mean,self.P_mean,self.T_mean,self.V_mean,self.D_var,self.P_var,self.T_var,self.V_var,
                 self.DP_cov,self.HP_cov,self.DH_cov,self.DT_cov,self.TP_cov,self.DH_corr,self.HP_corr,
                 self.DP_corr,self.DT_corr,self.TP_corr]
        plotlist2=[self.D2_mean,self.P2_mean,self.T2_mean,self.V2_mean,self.D2_var,self.P2_var,self.T2_var,self.V2_var,
                 self.DP2_cov,self.HP2_cov,self.DH2_cov,self.DT2_cov,self.TP2_cov,self.DH2_corr,self.HP2_corr,
                 self.DP2_corr,self.DT2_corr,self.TP2_corr]
    
        if canvas_n==f1:
            index=int(listbox.curselection()[0])
            IPM=plotlist[index]
            DPM=plotlist2[index]
            #f1=fig.add_subplot(111,title=self.listbox[index],xlabel='Distance (m)',ylabel='Mean')
        elif canvas_n==f2:
            index2=int(listbox2.curselection()[0])
            IPM=plotlist[index2]
            DPM=plotlist2[index2]
            #f2=fig2.add_subplot(111,title=self.listbox[index],xlabel='Distance (m)',ylabel='Mean')
        
        #-----------------------------------Parameter setting-------------------------------------
        legend=[]
        tc=['5','25','50','125','150','200','300','400','500','800']
        t=[0,0,0,0,0,0,0,0,0]
        I=["LPM 5a","LPM 25a","LPM 50a","LPM 125a","LPM 150a","LPM 200a","LPM 300a","LPM 400a","LPM 500a"]
        D=["NPM 5a","NPM 25a","NPM 50a","NPM 125a","NPM 150a","NPM 200a","NPM 300a","NPM 400a","NPM 500a"]
        color=['olive','g','darkorange','r','b','y']
        Isymbol=['o','s','<','d','^','o','s','<','d']
        Dsymbol=['-o','-s','-<','-d','-^','-o','-s','-<','-d']
        x=np.linspace(0,150,151)

        #------------------------------------Plotting(Spatial domain)---------------------------------------------- 
        for i in range(len(tc)):
            if t_list.count(tc[i])==1:
                t[i]=1
        count=0
        symbol=0
        for i in range(len(t)):
            if t[i]==1:
                canvas_n.plot(x,IPM[:,count],color=color[symbol],linestyle='-.',linewidth='2',markeredgecolor='k')
                legend.append(I[count])
                symbol=symbol+1
            count=count+1
        
        count=0
        symbol=0
        for item in t:
            if item==1:
                canvas_n.plot(x,DPM[:,count],Dsymbol[symbol],linewidth='1',color='k',markerfacecolor=color[symbol],markeredgecolor='k',markersize='5',markevery=3)
                legend.append(D[count])
                symbol=symbol+1
            count=count+1

        canvas_n.grid(alpha=0.3)
        canvas_n.legend(legend,ncol=2,fontsize='9',edgecolor='k')
        plt.rc('font', size=9)
        if canvas_n==f1:
            canvas.draw_idle()
        elif canvas_n==f2:
            canvas2.draw_idle()

    def button_event():
        import pandas as pd
        import os
        import numpy as np
        path1=myentry.get()
        path2=myentry2.get()
        figdir=myentry3.get()
        t_list=myentry4.get().split(',')

        if os.path.isdir(figdir):
            nothing=0
        else:
            os.mkdir(figdir)

        # Data path
        IPM_Folderpath=path1+'\\'
        DPM_Folderpath=path2+'\\'
        figdir=figdir+'\\'

        # File path for IPM
        D_path=IPM_Folderpath+'Displacement.txt'
        P_path=IPM_Folderpath+'Pressure.txt'
        T_path=IPM_Folderpath+'Temperature.txt'
        V_path=IPM_Folderpath+'Volumetric_strain.txt'

        Dt_path=IPM_Folderpath+'t_Displacement.txt'
        Pt_path=IPM_Folderpath+'t_Pressure.txt'
        Tt_path=IPM_Folderpath+'t_Temperature.txt'
        Vt_path=IPM_Folderpath+'t_Volumetric_strain.txt'
        # File path for DPM

        D2_path=DPM_Folderpath+'Displacement.txt'
        P2_path=DPM_Folderpath+'Pressure.txt'
        T2_path=DPM_Folderpath+'Temperature.txt'
        V2_path=DPM_Folderpath+'Volumetric_strain.txt'

        Dt2_path=DPM_Folderpath+'t_Displacement.txt'
        Pt2_path=DPM_Folderpath+'t_Pressure.txt'
        Tt2_path=DPM_Folderpath+'t_Temperature.txt'
        Vt2_path=DPM_Folderpath+'t_Volumetric_strain.txt'

        # File path for DPM Patameters
        Cp2_path=DPM_Folderpath+'Cp.txt'
        kT2_path=DPM_Folderpath+'kc.txt'
        n2_path=DPM_Folderpath+'n.txt'
        Hydraulic2_path=DPM_Folderpath+'Hydraulic.txt'


        # Data reading (IPM)
        self.D,self.P,self.T,self.V=Dt_Plot.Datareading(D_path),Dt_Plot.Datareading(P_path),Dt_Plot.Datareading(T_path),Dt_Plot.Datareading(V_path)
        self.Dt,self.Pt,self.Tt,self.Vt=Dt_Plot.Datareading(Dt_path),Dt_Plot.Datareading(Pt_path),Dt_Plot.Datareading(Tt_path),Dt_Plot.Datareading(Vt_path)

        # Data reading (DPM)
        self.D2,self.P2,self.T2,self.V2=Dt_Plot.Datareading(D2_path),Dt_Plot.Datareading(P2_path),Dt_Plot.Datareading(T2_path),Dt_Plot.Datareading(V2_path)
        self.Dt2,self.Pt2,self.Tt2,self.Vt2=Dt_Plot.Datareading(Dt2_path),Dt_Plot.Datareading(Pt2_path),Dt_Plot.Datareading(Tt2_path),Dt_Plot.Datareading(Vt2_path)

        #Data reading (DPM parameters)
        self.Cp2,self.kT2,self.n2,self.Hydraulic2=(Dt_Plot.Datareading(Cp2_path),Dt_Plot.Datareading(kT2_path),
                                                    Dt_Plot.Datareading(n2_path),Dt_Plot.Datareading(Hydraulic2_path))

        # IPM parameters
        self.Cp=pd.DataFrame((950*0.54+4200*0.46)*np.ones_like(self.Cp2))
        self.kT=pd.DataFrame((1.2*0.54+0.46*0.56)*np.ones_like(self.kT2))
        self.n=pd.DataFrame(0.46*np.ones_like(self.n2))
        self.Hydraulic=pd.DataFrame((3.5e-20*997.03*9.80667/8.9255e-4)*np.ones_like(self.Hydraulic2))


        #Differenece of IPM and DPM
        self.D_diff=(self.D2.iloc[:,1:]-self.D.iloc[:,1:])/self.D.iloc[:,1:]
        self.P_diff=(self.P2.iloc[:,1:]-self.P.iloc[:,1:])/self.P.iloc[:,1:]
        self.T_diff=(self.T2.iloc[:,1:]-self.T.iloc[:,1:])/self.T.iloc[:,1:]
        self.V_diff=(self.V2.iloc[:,1:]-self.V.iloc[:,1:])/self.V.iloc[:,1:]


        #Difference of IPM and DPM (Parameters)
        self.Cp_diff=(self.Cp2.iloc[:,1:])/self.Cp.iloc[:,1:]
        self.kT_diff=(self.kT2.iloc[:,1:])/self.kT.iloc[:,1:]
        self.n_diff=(self.n2.iloc[:,1:])/self.n.iloc[:,1:]
        self.Hydraulic_diff=(self.Hydraulic2.iloc[:,1:])/self.Hydraulic.iloc[:,1:]


        for i in range(1,len(self.T.columns),1):
            self.T.iloc[:,i]=self.T.iloc[:,i]-273.15
            self.T2.iloc[:,i]=self.T2.iloc[:,i]-273.15

        plt.figure(num=0,figsize=(9,7))
        Dt_Plot.Plotting(self.D,self.D2,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Displacement (m)',fontsize='28')
        plt.title("Host rock model (Spatial)",fontsize="28")
        plt.tight_layout()

        plt.figure(num=1,figsize=(9,7))
        Dt_Plot.Plotting(self.D,self.D2,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Displacement (m)',fontsize='28')
        plt.title("Host rock model (Spatial)",fontsize="28")
        plt.tight_layout()
        plt.savefig(figdir+'Displacement.png',dpi=500)

        plt.figure(num=2,figsize=(9,7))
        Dt_Plot.Plotting(self.P,self.P2,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Pressure (Pa)',fontsize='28')
        plt.title("Host rock model (Spatial)",fontsize="28")
        plt.tight_layout()
        plt.savefig(figdir+'Pressure.png',dpi=500)

        plt.figure(num=3,figsize=(9,7))
        Dt_Plot.Plotting(self.T,self.T2,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Temperature (℃)',fontsize='28')
        plt.title("Host rock model (Spatial)",fontsize="28")
        plt.ticklabel_format(style='plain',axis='y',scilimits=(0,0),useMathText=False,useOffset=False)
        plt.tight_layout()
        plt.savefig(figdir+'Temperature.png',dpi=500)


        plt.figure(num=4,figsize=(9,7))
        Dt_Plot.Plotting(self.V,self.V2,t_list)
        plt.grid(alpha=0.6)
        plt.ylabel('Volumetric strain (-)',fontsize='28')
        plt.title("Host rock model (Spatial)",fontsize="28")
        plt.tight_layout()
        plt.savefig(figdir+'VolumetricStrain.png',dpi=500)
        
        plt.close('all')


<h2> Tkinter (GUI) </h2>

In [95]:
class GUI():
             
    def __init__(self, root):
        self.root = root # root is a passed Tk object
        self.root.title("Unititled THM Project")
        self.x_resolution=win32api.GetSystemMetrics(win32con.SM_CXSCREEN)
        self.y_resolution=win32api.GetSystemMetrics(win32con.SM_CYSCREEN)
        padx= int((self.x_resolution-500)/2)
        pady= int((self.y_resolution-300)/2)
        self.root.geometry(f"+{padx}+{ pady}")
        self.root.geometry("500x300")
        self.root.resizable(width=0,height=0)
        self.tab_main=ttk.Notebook(self.root)
        self.tab_main.place(relx=0,rely=0,relwidth=1,relheight=1)
        style = th.ThemedStyle(self.root)
        style.set_theme("black")
        self.frame=ttk.Frame(self.root)
        self.frame.place(relx=0,rely=0,relwidth=1,relheight=1)
        self.button = ttk.Button(self.frame, text="Create new\n   project", command=self.create_new_project)
        self.button.place(x=100,y=50,height=100,width=100)
        self.button2 = ttk.Button(self.frame, text="Open project", command=lambda:self.load(0))
        self.button2.place(x=300,y=50,height=100,width=100)
        self.save_checker=True
        self.ProgressCounter=0
        self.PlotList=["Mean_Displacement","Mean_Pressure","Mean_Temperature","Mean_Volumetricstrain",
                  "C(u,u)","C(P,P)","C(v,v)","C(u,P)","C(y,P)","C(y,u)","C(u,T)","C(T,P)",
                 "Corr(u,u)","Corr(P,P)","Corr(v,v)","Corr(u,P)","Corr(y,P)","Corr(y,u)",
                 "Corr(u,T)","Corr(T,P)"]
        self.linecolor=['olive','g','darkorange','r','b','y']
        self.linesymbol=['-o','-s','-<','-d','-^','-p']
        self.linestyle=['-', '--', '-.', ':']
        self.linewidth_LPM=2
        self.linewidth_NPM=1
        self.linstyle_index=3
        self.Markersize=5
        self.Markevery=3
           
        #-----------------------------------------------MENU TOOLS-------------------------------------------------------
        menubar = tk.Menu(self.root)
        filemenu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label='File', menu=filemenu)
        filemenu.add_command(label='New', command=self.create_new_project)
        filemenu.add_command(label='Save as', command=self.save)
        filemenu.add_command(label='Load', command=lambda:self.load(1))
        editmenu=tk.Menu(menubar,tearoff=0)
        menubar.add_cascade(label="Edit",menu=editmenu)
        editmenu.add_command(label='Plot preference', command=self.Plot_Options)
        self.root.config(menu=menubar)

        global Case1,Case2
        Case1=MC_Plot()
        Case2=Dt_Plot()
        self.root.protocol("WM_DELETE_WINDOW", self.ShutDown)
    
    def tab_stochastic(self):
        
        padx= int((self.x_resolution-1400)/2)
        pady= int((self.y_resolution-950)/2-50)
        self.root.geometry(f"+{padx}+{pady}")
        self.root.geometry("1400x950")
        self.frame.destroy()
        
        try:
            self.tab1.destroy()
        except:
            print("Tab1 did not exisit")
        
        self.tab1=ttk.Frame(self.tab_main)
        self.tab1.place(x=0,y=30)
        
        self.Plot1=ttk.Notebook(self.tab1)
        self.Plot1.place(x=850,y=0,width=550,height=450)

        self.Plot2=ttk.Notebook(self.tab1)
        self.Plot2.place(x=850,y=460,width=550,height=450)
        
        global fig,fig2,canvas,canvas2

        fig = Figure(figsize=(2, 2), dpi=100,tight_layout=True)

        canvas = FigureCanvasTkAgg(fig, master=self.Plot1)  
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        toolbar = NavigationToolbar2Tk(canvas, self.Plot1)

        fig2 = Figure(figsize=(2, 2), dpi=100,tight_layout=True)

        canvas2 = FigureCanvasTkAgg(fig2, master=self.Plot2)  
        canvas2.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        toolbar2 = NavigationToolbar2Tk(canvas2, self.Plot2)

        self.tab_main.add(self.tab1,text='Stochastic')
        
        #------------------------------------Browse function------------------------------------------------------ 
        def browse_button_IPM():
            filename = tk.filedialog.askdirectory()
            IPMPathEntry.delete(0,tk.END)
            IPMPathEntry.insert(0,filename)

        def browse_button_DPM():
            filename = tk.filedialog.askdirectory()
            DPMPathEntry.delete(0,tk.END)
            DPMPathEntry.insert(0,filename)

        def browse_button_Save():
            filename = tk.filedialog.askdirectory()
            FigureSavePath.delete(0,tk.END)
            FigureSavePath.insert(0,filename)

        #------------------------------------Plot update function-------------------------------------------------

        def UpdatePlot(event):
            global f1
            fig.clear()

            P_title=Case1.listbox[int(listbox.curselection()[0])]
            f1=fig.add_subplot(111,title=P_title,xlabel='Distance (m)',ylabel=P_title)
            Case1.Cavanas_Plot(f1)

        def UpdatePlot2(event):
            global f2
            fig2.clear()

            P_title=Case1.listbox[int(listbox2.curselection()[0])]
            f2=fig2.add_subplot(111,title=P_title,xlabel='Distance (m)',ylabel=P_title)
            Case1.Cavanas_Plot(f2)

        
        #-----------------------------------------List Box and Text Box-------------------------------------------------------
        global listbox, listbox2, messagelistbox
        listbox=tk.Listbox(self.tab1)
        listbox.place(x=50,y=490,width=300,height=300)
        listbox.bind("<<ListboxSelect>>", UpdatePlot)
        
        listbox2=tk.Listbox(self.tab1)
        listbox2.place(x=400,y=490,width=300,height=300)
        listbox2.bind("<<ListboxSelect>>", UpdatePlot2)
        
        messagelistbox=tk.Text(self.tab1)
        messagelistbox.place(x=250,y=350,width=500,height=100)
        messagelistbox.bind("<Key>", lambda e: ctrlEvent(e))
        
        Scroll_message = tk.Scrollbar(messagelistbox)
        Scroll_message.pack(side="right", fill="y")
        
        Scroll_message.config(command=messagelistbox.yview)
        messagelistbox.config(yscrollcommand=Scroll_message.set)
       
        def ctrlEvent(event):
            if(12==event.state and event.keysym=='c' ):
                return
            else:
                return "break"

        #------------------------------------Options for Spatial Plotting---------------------------------
        #
        #
        #------------------------------------------Label-----------------------------------------------
        title=tk.Label(self.tab1,text="Spatial Domain Plotting",background='blue', foreground='white',font='18')
        title.grid(row=0,column=1)
        
        LPM_Label=ttk.Label(self.tab1,text='Path for LPM')
        LPM_Label.grid(row=1,column=0)

        NPM_Label=ttk.Label(self.tab1,text='Path for NPM')
        NPM_Label.grid(row=2,column=0)

        FigureLabel=ttk.Label(self.tab1,text='Path for figure saving')
        FigureLabel.grid(row=3,column=0)

        DpiLabel=ttk.Label(self.tab1,text='dpi')
        DpiLabel.place(x=630,y=135)

        TimeLabel=ttk.Label(self.tab1,text='Time')
        TimeLabel.grid(row=4,column=0)
        
        global Data_Label
        Data_Label=tk.Label(self.tab1,text='No data',bg='blue', fg='white')
        Data_Label.grid(row=6,column=0)
        
        global button_Parallel_label
        button_Parallel_label=tk.Label(self.tab1,text='Single-Core mode',bg='blue', fg='white')
        button_Parallel_label.grid(row=7,column=0,padx=5,ipadx=5)

        TimeLabel_2=ttk.Label(self.tab1,text='Format=5,25,50,125,150,200,300,400,500,800')
        TimeLabel_2.grid(row=4,column=2)
        
        nRLabel=ttk.Label(self.tab1,text="Number of Realizations")
        nRLabel.grid(row=5,column=0,padx=30,pady=15,sticky=tk.S)

        CoreLabel=ttk.Label(self.tab1,text='Core numbers')
        CoreLabel.grid(row=6,column=2,sticky=tk.S)

        CanvasLabel=ttk.Label(self.tab1,text="Plot 1",foreground='#00ffff')
        CanvasLabel.place(x=175,y=465)

        CanvasLabel2=ttk.Label(self.tab1,text="Plot 2",foreground='#00ffff')
        CanvasLabel2.place(x=530,y=465)
         #------------------------------------------Entry-----------------------------------------------
        global IPMPathEntry
        IPMPathEntry = ttk.Entry(self.tab1)
        IPMPathEntry.insert(0,'C:\JianYu\THM project\First_Year\Data(Results)\HostRock_Hydraulic_IPM_20210930_HydraulicRandomVariables')
        IPMPathEntry.grid(row=1,column=1,pady=15,ipadx=100)
        
        global DPMPathEntry
        DPMPathEntry=ttk.Entry(self.tab1)
        DPMPathEntry.insert(0,'C:\JianYu\THM project\First_Year\Data(Results)\HostRock_Hydraulic_DPM_HydraulicRandomVarialbes_20210930')
        DPMPathEntry.grid(row=2,column=1,pady=15,ipadx=100)
        
        global FigureSavePath
        FigureSavePath=ttk.Entry(self.tab1)
        FigureSavePath.insert(0,'C:\JianYu\THM project\GUITEST\\Stochastic')
        FigureSavePath.grid(row=3,column=1,pady=15,ipadx=100)
        
        global Figuredpi
        Figuredpi=ttk.Entry(self.tab1)
        Figuredpi.insert(0,"300")
        Figuredpi.grid(row=3,column=2,sticky=tk.E)
        
        global TimeEntry
        TimeEntry=ttk.Entry(self.tab1)
        TimeEntry.insert(0,'25,50,150,300')
        TimeEntry.grid(row=4,column=1,pady=15,ipadx=100)
        
        global nREntry
        nREntry=ttk.Entry(self.tab1)
        nREntry.insert(0,'8000')
        nREntry.grid(row=5,column=1,padx=30,pady=15)

        #------------------------------------------Button-----------------------------------------------
        global mybutton_Data
        mybutton_Data = ttk.Button(self.tab1, text="Run", command=lambda:[GUI_thread.thread(Case1.button_event_Data),GUI_thread.thread(Case1.Progress)])
        mybutton_Data.grid(row=6,column=1,padx=25,pady=10,ipadx=32,sticky=tk.W)

        Browse_button_IPM = ttk.Button(self.tab1,text="Browse", command=browse_button_IPM)
        Browse_button_IPM.grid(row=1, column=2,sticky=tk.W)

        Browse_button_DPM = ttk.Button(self.tab1,text="Browse", command=browse_button_DPM)
        Browse_button_DPM.grid(row=2, column=2,sticky=tk.W)

        Browse_button_Save = ttk.Button(self.tab1,text="Browse", command=browse_button_Save)
        Browse_button_Save.grid(row=3, column=2,sticky=tk.W)
        
        global mybutton_Plot
        #mybutton_Plot= ttk.Button(self.tab1, text="Save Plot", command=lambda:GUI_thread.thread(Case1.button_event_Plot))
        #mybutton_Plot.grid(row=6,column=1,padx=26,pady=10,ipadx=33,sticky=tk.E)
        
        mybutton_Plot= ttk.Button(self.tab1, text="Plot", command=lambda:[Case1.button_event_Plot,self.Progress_Windows])
        mybutton_Plot.grid(row=6,column=1,padx=25,pady=10,ipadx=34,sticky=tk.E)

        global mybutton_Parallel
        mybutton_Parallel= ttk.Button(self.tab1, text="Engine Start", command=lambda: GUI_thread.thread(Case1.button_Parallel))
        mybutton_Parallel.grid(row=7,column=1,padx=25,ipadx=27,sticky=tk.W)
        
        global mybutton_Parallel2
        mybutton_Parallel2= ttk.Button(self.tab1, text="Engine Stop", command=Case1.button_Parallel_stop)
        mybutton_Parallel2.grid(row=7,column=1,padx=25,ipadx=27,sticky=tk.E)

        #Test_button=ttk.Button(self.tab1, text="Test",command=self.Progress_Windows)
        #Test_button.grid(row=8,column=2)
        
        global number,Select
        number = tk.StringVar()
        Select= ttk.Combobox(self.tab1, width=12, textvariable=number, state='readonly')
        Select['values'] = ["1"] 
        Select.grid(row=7,column=2)      
        Select.current(0)   

        #K_check=tk.BooleanVar();K_check.set(True)
        #kc_check=tk.BooleanVar();kc_check.set(False)
        #Cp_check=tk.BooleanVar();Cp_check.set(False)
        #aT_check=tk.BooleanVar();aT_check.set(False)
        #E_check=tk.BooleanVar();E_check.set(False)
       
        global RV_check
        RV_check=tk.IntVar();

        K_checkbox=ttk.Radiobutton(self.tab1,text="Hydraulic conductivity",var=RV_check,value=0)
        K_checkbox.grid(row=9,column=0)

        kc_checkbox=ttk.Radiobutton(self.tab1,text="Thermal conductivity",var=RV_check,value=1)
        kc_checkbox.grid(row=10,column=0)

        Cp_checkbox=ttk.Radiobutton(self.tab1,text="Specific heat",var=RV_check,value=2)
        Cp_checkbox.grid(row=11,column=0)

        aT_checkbox=ttk.Radiobutton(self.tab1,text="Thermal expansion",var=RV_check,value=3)
        aT_checkbox.grid(row=12,column=0)

        E_checkbox=ttk.Radiobutton(self.tab1,text="Young's modulus",var=RV_check,value=4)
        E_checkbox.grid(row=13,column=0)

        #mybutton_Plot['state']="disabled"
        
    def tab_deterministic(self):
        
        self.frame.destroy()
        
        try:
            self.tab2.destroy()
        except:
            print("Tab2 did not exisit")
        
        self.tab2=ttk.Frame(self.tab_main)
        self.tab2.place(x=10,y=30)
        self.tab_main.add(self.tab2,text='Deterministic')

        self.Plot3=ttk.Notebook(self.tab2)
        self.Plot3.place(x=850,y=0,width=550,height=450)

        self.Plot4=ttk.Notebook(self.tab2)
        self.Plot4.place(x=850,y=460,width=550,height=450)

        global fig3,fig4,canvas3,canvas4

        fig3 = Figure(figsize=(2, 2), dpi=100,tight_layout=True)

        canvas3 = FigureCanvasTkAgg(fig3, master=self.Plot3)  
        canvas3.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        toolbar3 = NavigationToolbar2Tk(canvas3, self.Plot3)

        fig4 = Figure(figsize=(2, 2), dpi=100,tight_layout=True)

        canvas4 = FigureCanvasTkAgg(fig4, master=self.Plot4)  
        canvas4.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        toolbar4 = NavigationToolbar2Tk(canvas4, self.Plot4)

        global listbox3,listbox4
        listbox3=tk.Listbox(self.tab2)
        listbox3.place(x=50,y=490,width=300,height=300)

        listbox4=tk.Listbox(self.tab2)
        listbox4.place(x=400,y=490,width=300,height=300)
        
        #Case2=Dt_Plot()

        title=tk.Label(self.tab2,text="Spatial Domain Plotting",bg='blue', fg='white',font='18')
        title.grid(row=0,column=1)

        label1=ttk.Label(self.tab2,text='Path for LPM')
        label1.grid(row=1,column=0)

        label2=ttk.Label(self.tab2,text='Path for NPM')
        label2.grid(row=2,column=0)

        label3=ttk.Label(self.tab2,text='Path for figure saving')
        label3.grid(row=3,column=0)

        label4=ttk.Label(self.tab2,text='Time')
        label4.grid(row=4,column=0)

        label4_2=ttk.Label(self.tab2,text='Format=5,25,50,125,150,200,300,400,500,800')
        label4_2.grid(row=4,column=3)
        
        global myentry
        myentry = ttk.Entry(self.tab2)
        myentry.insert(0,'C:\JianYu\THM project\First_Year\Data(Results)\HostRock(Test)\IPM')
        myentry.grid(row=1,column=1,pady=15,ipadx=100)
        
        global myentry2
        myentry2=ttk.Entry(self.tab2)
        myentry2.insert(0,'C:\JianYu\THM project\First_Year\Data(Results)\HostRock(Test)\DPM')
        myentry2.grid(row=2,column=1,pady=15,ipadx=100)
        
        global myentry3
        myentry3=ttk.Entry(self.tab2)
        myentry3.insert(0,'C:\JianYu\THM project\GUITEST')
        myentry3.grid(row=3,column=1,pady=15,ipadx=100)
        
        global myentry4
        myentry4=ttk.Entry(self.tab2)
        myentry4.insert(0,'25,50,150,300')
        myentry4.grid(row=4,column=1,pady=15,ipadx=100)

        mybutton = ttk.Button(self.tab2, text='Run', command=lambda:GUI_thread.thread(Dt_Plot.button_event))
        mybutton.grid(row=5,column=1)

        #---------------------------Options for Temporal Plotting--------------------------------------

        title=tk.Label(self.tab2,text="Temporal Domain Plotting",bg='orange', fg='black',font="20")
        title.grid(row=6,column=1,pady=15)

        t_label=ttk.Label(self.tab2,text='Path for LPM')
        t_label.grid(row=7,column=0)

        t_label2=ttk.Label(self.tab2,text='Path for NPM')
        t_label2.grid(row=8,column=0)

        t_label3=ttk.Label(self.tab2,text='Path for figure saving')
        t_label3.grid(row=9,column=0)

        t_plot=ttk.Entry(self.tab2)
        t_plot.insert(0,"C:\JianYu\THM project\First_Year\Data(Results)\HostRock(Test)\\IPM")
        t_plot.grid(row=7,column=1,pady=10,ipadx=100)

        t_plot2=ttk.Entry(self.tab2)
        t_plot2.insert(0,"C:\JianYu\THM project\First_Year\Data(Results)\HostRock(Test)\\DPM")
        t_plot2.grid(row=8,column=1,pady=10,ipadx=100)

        t_plot3=ttk.Entry(self.tab2)
        t_plot3.insert(0,"C:\JianYu\THM project\GUITEST")
        t_plot3.grid(row=9,column=1,pady=10,ipadx=100)

        mybutton2 = ttk.Button(self.tab2, text='Run', command=lambda:MC_plot.button_event2)
        mybutton2.grid(row=10,column=1)
        
                
    def save(self):

        path=tk.filedialog.asksaveasfilename(filetypes=[("Pickle Dumps","*.pkl")],defaultextension='.pickle')
        
        with open(path, 'wb') as f:
            pickle.dump(Case1, f)
            
        f.close()
        self.save_checker=True
        
    def load(self,discriminator):

        path=tk.filedialog.askopenfilename()
        global Case1
        with open(path, 'rb') as f:
            
            Case1=pickle.load(f)
        
        f.close()
        
        if discriminator==0:
            self.frame.destroy()
        if discriminator==1:
            self.tab_main.destroy()
            self.tab_main=ttk.Notebook(self.root)
            self.tab_main.place(relx=0,rely=0,relwidth=1,relheight=1)

        filename=path.split("/")[-1]
        filename=filename.split(".")[0]
        
        self.tab_stochastic()
        self.root.title(filename)
        messagelistbox.insert(tk.END,"Load file:"+filename+" ("+str(time.asctime())+")\n")
        
        
        for item in self.PlotList:
            listbox.insert(tk.END,item)
            listbox2.insert(tk.END,item)
            Case1.listbox.append(item)

    def ShutDown(self):
        import threading
        if self.save_checker==True:
            warning=tk.messagebox.askquestion(title = 'Warning',message="Leave?")
            if warning=='no':
                return 0
            else:
                subprocess.Popen(["ipcluster", "stop"])
                try:
                    self.root.destroy()
                except OSError:
                    print('Shut Down')
        else:
            warning=tk.messagebox.askyesnocancel(title = 'Warning',message="Save file before leave?")
            print(warning)
            if warning==False:
                subprocess.Popen(["ipcluster", "stop"])
                try:
                    self.root.destroy()
                except OSError:
                    print('Shut Down')
            elif warning==True:
                self.save()
                subprocess.Popen(["ipcluster", "stop"])
                try:
                    self.root.destroy()
                except OSError:
                    print('Shut Down')
            else:
                return 0
            
    def Progress_Windows(self):
        
        progressWindow = tk.Toplevel(self.root)
        progressWindow.title("Progress")
        
        global progressFrame
        progressFrame=ttk.Frame(progressWindow)
        progressFrame.place(relx=0,rely=0,relwidth=1,relheight=1)
        toplevel_offsetx, toplevel_offsety = self.root.winfo_x() + self.root.winfo_width(), self.root.winfo_y()
        padx= -1000
        pady= 400
        
        progressWindow.geometry(f"+{toplevel_offsetx + padx}+{toplevel_offsety + pady}")
        progressWindow.geometry("600x150")
        pstyle = th.ThemedStyle(progressFrame)
        pstyle.configure("red.Horizontal.TProgressbar", troughcolor ='gray', background='green2',foreground='red')
        global progress
        progress=ttk.Progressbar(progressFrame,orient=tk.HORIZONTAL,length=500,mode='determinate', style="red.Horizontal.TProgressbar")
        progress_label=ttk.Label(progressFrame,text="Plotting....")
        progress_label.place(x=35,y=30)
        progress.place(x=50,y=60)

    def Plot_Options(self):
        
        def close():
            self.root.attributes("-disabled",0)
            OptionsWindow.destroy()
            
        def Update_PreViewPlot(canvas_n):
            if canvas_n==Prefig:
                Prefig.clear()
            else:
                Prefig2.clear()
            Pref1=canvas_n.add_subplot(111,title="PreView",xlabel='Distance',ylabel="PreView")
            
            for i in range(len(self.linecolor)):
                x=np.linspace(0,10,11)
                y=np.ones_like(x)*i
                y2=y-0.5
                Line_color=self.linecolor[i]
                if Line_color.count(',')>0:
                    Line_color=Line_color.split(',')
                    Line_color=[float(x)/255 for x in Line_color]
                Pref1.plot(x,y,self.linesymbol[i],color='k',linewidth=self.linewidth_NPM,markerfacecolor=Line_color,markeredgecolor='k',
                          markersize=self.Markersize,markevery=self.Markevery)
                Pref1.plot(x,y2,self.linestyle[self.linstyle_index],color=Line_color,linewidth=self.linewidth_LPM)
                
                Pref1.text(10.5,i-0.1,"NPM-"+str(i),color='r')
                Pref1.text(10.5,i-0.6,"LPM-"+str(i),color='b')
            if canvas_n==Prefig:
                Precanvas.draw_idle()
            else:
                Precanvas2.draw_idle()
                
        def Apply_Marker():
            
            self.Markersize=int(Markersize_entry.get())
            self.Markevery=int(Markevery_entry.get())
            self.linstyle_index=int(Linestyle_select.current())
            self.linewidth_LPM=int(Linewidth_LPM_entry.get())
            self.linewidth_NPM=int(Linewidth_NPM_entry.get())
            
            Case1.Markersize=self.Markersize
            Case1.Markevery=self.Markevery
            Case1.linstyle_index=self.linstyle_index
            Case1.linewidth_LPM=self.linewidth_LPM
            Case1.linewidth_NPM=self.linewidth_NPM
            
        def Adjust_color():
            
            def adjustclose():
                AdjustWindow.destroy()
            def adjustok():
                color_var=Value_entry.get()
                index=LineColorlistbox.curselection()[0]
                self.linecolor[index]=color_var
                LineColorlistbox.insert(index,color_var)
                LineColorlistbox.delete(index+1)
                AdjustWindow.destroy()
                
                Case1.linecolor=self.linecolor
                          
            def display_palette():
                (rgb,hx)=colorchooser.askcolor(parent=AdjustWindow)
                var = tk.StringVar()
                var.set(hx)
                r=hx[1:3]
                g=hx[3:5]
                b=hx[5:7]
                Color_str=str(int(r,16))+','+str(int(g,16))+','+str(int(b,16))
                Value_entry.insert(0,Color_str)
                
            if not LineColorlistbox.curselection():
                error=tk.messagebox.showerror(parent=OptionsWindow,title = 'Error',message="Please select one Line color object")
                return 0
           
            AdjustWindow=tk.Toplevel(OptionsWindow)
            AdjustFrame=ttk.Frame(AdjustWindow)
            AdjustFrame.place(relx=0,rely=0,relwidth=1,relheight=1)
            offsetx, offsety = OptionsWindow.winfo_x() + OptionsWindow.winfo_width(), OptionsWindow.winfo_y()
            padx= -600
            pady= 180
            AdjustWindow.geometry(f"+{offsetx + padx}+{offsety + pady}")
            AdjustWindow.geometry("350x150")
            
            Value_Label=ttk.Label(AdjustFrame,text="Input color")
            Value_Label.place(x=20,y=50)
            
            Value_entry=ttk.Entry(AdjustFrame)
            Value_entry.place(x=100,y=50)
            
            Value_OK_button=ttk.Button(AdjustFrame,text="OK",command=adjustok)
            Value_OK_button.place(x=70,y=100)
            
            Value_Cancel_button=ttk.Button(AdjustFrame,text="Cancel",command=adjustclose)
            Value_Cancel_button.place(x=190,y=100)
            
            Palatte_button=ttk.Button(AdjustFrame,text="Palette",command=display_palette)
            Palatte_button.place(x=250,y=47)
            
                
        def Adjust_symbol():
            def adjustclose():
                AdjustWindow.destroy()
            def adjustok():
                symbol_var=Value_entry.get()
                index=Linesymbollistbox.curselection()[0]
                self.linesymbol[index]=symbol_var
                Linesymbollistbox.insert(index,symbol_var)
                Linesymbollistbox.delete(index+1)
                AdjustWindow.destroy()
                
                Case1.linesymbol=self.linesymbol
                
            if not  Linesymbollistbox.curselection():
                error=tk.messagebox.showerror(parent=OptionsWindow,title = 'Error',message="Please select one Line style object")
                return 0
                
            AdjustWindow=tk.Toplevel(OptionsWindow)
            AdjustFrame=ttk.Frame(AdjustWindow)
            AdjustFrame.place(relx=0,rely=0,relwidth=1,relheight=1)
            offsetx, offsety = OptionsWindow.winfo_x() + OptionsWindow.winfo_width(), OptionsWindow.winfo_y()
            padx= -600
            pady= 180
            AdjustWindow.geometry(f"+{offsetx + padx}+{offsety + pady}")
            AdjustWindow.geometry("350x150")
            
            Value_Label=ttk.Label(AdjustFrame,text="Input color")
            Value_Label.place(x=20,y=50)
            
            Value_entry=ttk.Entry(AdjustFrame)
            Value_entry.place(x=100,y=50)
            
            Value_OK_button=ttk.Button(AdjustFrame,text="OK",command=adjustok)
            Value_OK_button.place(x=70,y=100)
            
            Value_Cancel_button=ttk.Button(AdjustFrame,text="Cancel",command=adjustclose)
            Value_Cancel_button.place(x=190,y=100)
            
            
        self.root.attributes("-disabled",1)
        OptionsWindow=tk.Toplevel(self.root)
        OptionsWindow.title("Plot preference")
        #padx= int(-toplevel_offsetx)
        #pady= int(toplevel_offsety)
        padx= int((self.x_resolution-900)/2)
        pady= int((self.y_resolution-500)/2-50)
        #self.root.geometry(f"+{padx}+{pady}")
        OptionsWindow.geometry(f"+{ padx}+{ pady}")
        OptionsWindow.geometry("900x500")
        OptionsWindow.protocol("WM_DELETE_WINDOW", close)
        
        Optionstab_main=ttk.Notebook(OptionsWindow)
        Optionstab_main.place(relx=0,rely=0,relwidth=1,relheight=1)
        
        Line_tab=ttk.Frame(Optionstab_main)
        Line_tab.place(x=0,y=30)
        Optionstab_main.add(Line_tab,text='Line options')
        
        Marker_tab=ttk.Frame(Optionstab_main)
        Marker_tab.place(x=10,y=30)
        Optionstab_main.add(Marker_tab,text='Marker options')
        
        #--------------------------------------Line tab-------------------------------------------------------------------
                
        PrePlot1=ttk.Notebook(Line_tab)
        PrePlot1.place(x=900-550,y=25,width=500,height=400)
        
        global Pref1,Prefig,Prefig2,Precanvas,Precanvas2

        Prefig = Figure(figsize=(2, 2), dpi=100,tight_layout=True)

        Precanvas = FigureCanvasTkAgg(Prefig, master=PrePlot1)  
        Precanvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        Pretoolbar = NavigationToolbar2Tk(Precanvas, PrePlot1)
        
        Listcolor_label=ttk.Label(Line_tab,text="Line color")
        Listcolor_label.place(x=50+18,y=70)
        
        Listsymbol_label=ttk.Label(Line_tab,text="Line symbol")
        Listsymbol_label.place(x=50+18+100+25,y=70)
        
        Adjustbutton=ttk.Button(Line_tab,text='Adjust color',command=Adjust_color)
        Adjustbutton.place(x=50+8,y=100+200+5)
        
        Adjustbutton2=ttk.Button(Line_tab,text='Adjust style',command=Adjust_symbol)
        Adjustbutton2.place(x=50+100+40,y=100+200+5)
        
        PreViewbutton=ttk.Button(Line_tab,text="Plot",command=lambda:Update_PreViewPlot(Prefig))
        PreViewbutton.place(x=50+75,y=350)
        
        
        global LineColorlistbox, Linesymbollistbox
        LineColorlistbox=tk.Listbox(Line_tab)
        LineColorlistbox.place(x=50,y=100,width=100,height=200)
        #LineColorlistbox.bind("<<ListboxSelect>>", UpdatePlot)
        
        Linesymbollistbox=tk.Listbox(Line_tab)
        Linesymbollistbox.place(x=180,y=100,width=100,height=200)
        #LineStylelistbox.bind("<<ListboxSelect>>", UpdatePlot2)
        
        #----------------------------------Marker tab----------------------------------------------
        
        PrePlot2=ttk.Notebook(Marker_tab)
        PrePlot2.place(x=900-550,y=25,width=500,height=400)

        Prefig2 = Figure(figsize=(2, 2), dpi=100,tight_layout=True)

        Precanvas2 = FigureCanvasTkAgg(Prefig2, master=PrePlot2)  
        Precanvas2.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        Pretoolbar2 = NavigationToolbar2Tk(Precanvas2, PrePlot2)
        
        Markersize_label=ttk.Label(Marker_tab,text='Markersize')
        Markersize_label.place(x=20,y=50)
        
        Markevery_label=ttk.Label(Marker_tab,text='Markevery')
        Markevery_label.place(x=20,y=100)
        
        Linewidth_LPM_label=ttk.Label(Marker_tab,text="LineWidth(LPM)")
        Linewidth_LPM_label.place(x=20,y=150)
        
        Linewidth_NPM_label=ttk.Label(Marker_tab,text="LineWidth(NPM)")
        Linewidth_NPM_label.place(x=20,y=200)
        
        Linestyle_LPM_label=ttk.Label(Marker_tab,text="LinStyle(LPM)")
        Linestyle_LPM_label.place(x=20,y=250)
        
        Markersize_entry=ttk.Entry(Marker_tab)
        Markersize_entry.insert(0,"5")
        Markersize_entry.place(x=150,y=50)
        
        Markevery_entry=ttk.Entry(Marker_tab)
        Markevery_entry.insert(0,"3")
        Markevery_entry.place(x=150,y=100)
        
        Linewidth_LPM_entry=ttk.Entry(Marker_tab)
        Linewidth_LPM_entry.insert(0,"2")
        Linewidth_LPM_entry.place(x=150,y=150)
        
        Linewidth_NPM_entry=ttk.Entry(Marker_tab)
        Linewidth_NPM_entry.insert(0,"1")
        Linewidth_NPM_entry.place(x=150,y=200)
        
        stylenumber=tk.StringVar()
        Linestyle_select=ttk.Combobox(Marker_tab,width=15,textvariable=stylenumber,state='readonly')
        Linestyle_select['values']=self.linestyle
        Linestyle_select.place(x=150,y=250)
        Linestyle_select.current(3)
        
        PreViewbutton=ttk.Button(Marker_tab,text="Plot",command=lambda:Update_PreViewPlot(Prefig2))
        PreViewbutton.place(x=50+75,y=350)
        
        Applybutton_tab=ttk.Button(Marker_tab,text="Apply",command=Apply_Marker)
        Applybutton_tab.place(x=50+75,y=325)
        
        for item in self.linesymbol:
            Linesymbollistbox.insert(tk.END,item)
        for item in self.linecolor:
            LineColorlistbox.insert(tk.END,item)
            
    
    def create_new_project(self):
        
        def close():
            self.root.attributes("-disabled",0)
            newWindow.destroy()
            
        def createproject():
            self.root.attributes("-disabled",0)
            newWindow.destroy()
            if Stochastic_checkvalue.get()==True:
                self.save_checker=False
                self.tab_stochastic()
            if Deterministic_checkvalue.get()==True:
                self.tab_deterministic()
        
        self.root.attributes("-disabled",1)
        
        newWindow = tk.Toplevel(self.root)
        newFrame=ttk.Frame(newWindow)
        newFrame.place(relx=0,rely=0,relwidth=1,relheight=1)
        toplevel_offsetx, toplevel_offsety = self.root.winfo_x() + self.root.winfo_width(), self.root.winfo_y()
        padx= -350
        pady= 100
        newWindow.geometry(f"+{toplevel_offsetx + padx}+{toplevel_offsety + pady}")
        newWindow.geometry("200x150")
        newWindow.protocol("WM_DELETE_WINDOW", close)
        
        Stochastic_checkvalue = tk.BooleanVar() 
        Stochastic_checkvalue.set(False)
        
        Deterministic_checkvalue=tk.BooleanVar()
        Deterministic_checkvalue.set(False)
        
        Stochastic = ttk.Checkbutton(newFrame, text='Stochastic', var= Stochastic_checkvalue) 
        Stochastic.place(x=30,y=25)        
        Deterministic= ttk.Checkbutton(newFrame, text='Deterministic', var=Deterministic_checkvalue) 
        Deterministic.place(x=30,y=50)
        
        OKbutton=ttk.Button(newFrame,text="OK",command=createproject)
        OKbutton.place(x=20,y=110)
        
        Cancelbutton=ttk.Button(newFrame,text="Cancel",command=close)
        Cancelbutton.place(x=100,y=110)
        
if __name__ == '__main__':             
    root = tk.Tk()

    W1=GUI(root)

    root.mainloop()

Tab1 did not exisit


<ipython-input-91-43d187b9037b>:115: RuntimeWarning: invalid value encountered in true_divide
  A[:,i]=np.mean((Y1[:,nR*i:nR*(i+1)]-temp1)*(Y2[:,nR*i:nR*(i+1)]-temp2),axis=1)/np.std(Y1[:,nR*i:nR*(i+1)],axis=1)/np.std(Y2[:,nR*i:nR*(i+1)],axis=1)
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\3002shinning\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-95-8a83180f611b>", line 125, in UpdatePlot
    P_title=Case1.listbox[int(listbox.curselection()[0])]
IndexError: tuple index out of range
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\3002shinning\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-95-8a83180f611b>", line 125, in UpdatePlot
    P_title=Case1.listbox[int(listbox.curselection()[0])]
IndexError: tuple index out of range


False


(None, None)

In [54]:
from tkinter import *
from tkinter.ttk import *

tk=Tk()
progress=Progressbar(tk,orient=HORIZONTAL,length=100,mode='determinate')

def bar():
    import time
    progress['value']=20
    tk.update_idletasks()
    time.sleep(1)
    progress['value']=50
    tk.update_idletasks()
    time.sleep(1)
    progress['value']=80
    tk.update_idletasks()
    time.sleep(1)
    progress['value']=100

progress.pack()
Button(tk,text='foo',command=bar).pack()
mainloop()

In [25]:
from tkinter import Tk, Frame, Button, Label

class GUI:

    def __init__(self, root):
        self.root = root # root is a passed Tk object
        self.button = Button(self.root, text="Push me", command=self.removethis)
        self.button.pack()
        self.frame = Frame(self.root)
        self.frame.pack()
        self.label = Label(self.frame, text="I'll be destroyed soon!")
        self.label.pack()

    def removethis(self):
        self.frame.destroy()

root = Tk()
root2 = Tk()
window = GUI(root)
window2=GUI(root2)

root.mainloop()

ModuleNotFoundError: No module named 'PySimpleGUI'

In [19]:
from tkinter import *
from tkinter import messagebox
from tkinter.colorchooser import *
root = Tk()
root.geometry("360x360+150+150")
root.update()
x = root.winfo_x()
y = root.winfo_y()
var = StringVar()
#用toplevel调整调色盘
t1 = Toplevel()
t1.geometry("+%d+%d" %(x+250,y+100))
#隐藏toplevel
t1.withdraw()
#显示调色盘
def display_palette():
    (rgb,hx)=askcolor(parent=t1)
    root.config(bg=hx)
    var.set(hx)
    h1=hx[1:3]
    h2=hx[3:5]
    h3=hx[5:7]
    s1.set(int(h1,16))
    s2.set(int(h2,16))
    s3.set(int(h3,16))
#通过尺度条刻度值调整颜色
def updatecolor_scale(args):
    red = s1.get()
    green = s2.get()
    blue = s3.get()
    mycolor="#%02x%02x%02x"%(red,green,blue)
    root.config(bg=mycolor)
    var.set(mycolor)
#通过输入值调整颜色
def updatecolor_entry(args):
    hexcolor = var.get()
    if not hexcolor.startswith('#',0,1):
        messagebox.showwarning(message='请以#开始')
    if len(hexcolor) != 7:
        messagebox.showwarning(message='输入数值在"#000000-#FFFFFF"之间')
    try:
        h1=hexcolor[1:3]
        h2=hexcolor[3:5]
        h3=hexcolor[5:7]
        s1.set(int(h1,16))
        s2.set(int(h2,16))
        s3.set(int(h3,16))
        root.config(bg=hexcolor)
    except:
        root.config(bg='#FFFFFF')
        s1.set(255)
        s2.set(255)
        s3.set(255)
#设计尺度条定位初始数值
s1 = Scale(root,from_=0,to=255,length=230,command=updatecolor_scale,orient='horizontal')
s2 = Scale(root,from_=0,to=255,length=230,command=updatecolor_scale,orient='horizontal')
s3 = Scale(root,from_=0,to=255,length=230,command=updatecolor_scale,orient='horizontal')
s1.set(255)
s2.set(255)
s3.set(255)
#通过尺度条数值转换成16进制颜色表示
hexs1=hex(s1.get()).lstrip('0x')
hexs2=hex(s2.get()).lstrip('0x')
hexs3=hex(s3.get()).lstrip('0x')
var.set('#'+hexs1+hexs2+hexs3)
l=Label(root,text='color(16)请输入:')
e=Entry(root,width=20,textvariable=var)
e.bind('<Return>',updatecolor_entry)
b=Button(root,text='显示调色盘',command=display_palette)
#各主键定位
s1.grid(row=0,columnspan=2,sticky='w')
s2.grid(row=1,columnspan=2,sticky='w')
s3.grid(row=2,columnspan=2,sticky='w')
l.grid(row=3,column=0,sticky='w')
e.grid(row=3,column=1,sticky='w')
b.grid(row=4,sticky='w')
root.mainloop()